In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-23 13:17:55,203 - NumExpr defaulting to 8 threads.


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2823618064736) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2823618067328) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2823618066320) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2823618067472) was called 1 times
Running: Function "load_module_lookup" (id=2823618068048) was called 1 times
Running: Function "load_ID_lookup" (id=2823618066752) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [69]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [ ]:
def duplicate_long_faults(fa_floor):
    fa_floor['Hours'] = (fa_floor['Duration']/3600)

In [70]:
fa_sel = feat.get_data_faults(fa_floor, modules=[1],PTT = 'PTT011')

#fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Both','Blue','Grey']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'Asset Code', agg_type = 'count')

Running: Function "load_PTT_lookup" (id=2823618066464) was called 254 times


In [10]:
av_sel,at_sel = feat.av_at_select(av, at, availability_select_options = {'Pick Station' : ['PTT011']}, remove_high_AT = True, AT_limit = None)

#av_sel,at_sel = feat.av_at_select(av, at, remove_high_AT = True, AT_limit = None)

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

#at_agg = at_agg[at_agg['TOTES']>100]

Running: Function "aggregate_availability" (id=2823618066608) was called 1 times


In [11]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = 'Module')

In [84]:
def create_PPT_df(fa_floor,at,av):
    
    pick_stations = ['PTT011','PTT012','PTT021','PTT022','PTT031','PTT032','PTT041','PTT042','PTT051','PTT052','PTT071','PTT072','PTT081','PTT082','PTT091','PTT092','PTT101','PTT102','PTT111','PTT112','PTT121','PTT122','PTT131','PTT132','PTT141','PTT142','PTT151','PTT152','PTT171','PTT172','PTT181','PTT182','PTT191','PTT192','PTT201','PTT202']
    df_PTT = pd.DataFrame()
    
    for PTT in pick_stations:
        
        module = int(PTT[3:5])
        
        fa_floor = fa_floor.copy()
        
        fa_floor['Duration'] = np.log10(fa_floor['Duration']) + 1
        
        fa_sel = feat.get_data_faults(fa_floor, modules=[module],PTT = PTT)                                                        
        fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'Asset Code', agg_type = 'sum')

        av_sel,at_sel = feat.av_at_select(av, at, availability_select_options = {'Pick Station' : [PTT]}, remove_high_AT = False, AT_limit = 60)
        av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
        at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

        df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = 'Module')
        
        df_PTT = pd.concat([df_PTT,df],axis=0,join='outer')
    
    df_PTT = df_PTT.fillna(0) 
    
    totes_col = df_PTT.pop('TOTES')
    df_PTT['TOTES'] = totes_col
    
    return df_PTT

In [85]:
df = create_PPT_df(fa_floor,at,av)

Running: Function "load_PTT_lookup" (id=2823618066464) was called 327 times
Running: Function "aggregate_availability" (id=2823618066608) was called 326 times
Running: Function "load_PTT_lookup" (id=2823618066464) was called 328 times
Running: Function "aggregate_availability" (id=2823618066608) was called 327 times
Running: Function "load_PTT_lookup" (id=2823618066464) was called 329 times
Running: Function "aggregate_availability" (id=2823618066608) was called 328 times
Running: Function "load_PTT_lookup" (id=2823618066464) was called 330 times
Running: Function "aggregate_availability" (id=2823618066608) was called 329 times
Running: Function "load_PTT_lookup" (id=2823618066464) was called 331 times
Running: Function "aggregate_availability" (id=2823618066608) was called 330 times
Running: Function "load_PTT_lookup" (id=2823618066464) was called 332 times
Running: Function "aggregate_availability" (id=2823618066608) was called 331 times
Running: Function "load_PTT_lookup" (id=282361

In [86]:
df

,timestamp,Downtime,Blue Tote Loss,Grey Tote Loss,C05,C0501ACH011,C0501RDC004,C0501STA002,C0501STA012,C0501STA020,...,SCS202,SCS203,SCS204,SCS205,SCS206,SCSM20,C1404RDC146,C1404STA149,C14PTT202,TOTES
0,2020-11-20 00:00:00,0.183832,0.183832,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.220339
1,2020-11-20 01:00:00,0.193668,0.193668,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.550000
2,2020-11-20 02:00:00,0.685402,0.572000,0.113402,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.633333
3,2020-11-20 03:00:00,0.032177,0.032177,0.000000,0.0,0.0,0.0,0.0,1.845098,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.533333
4,2020-11-20 04:00:00,0.504636,0.504636,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,2020-12-09 13:00:00,0.010749,0.006667,0.004083,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,1.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,49.266667
470,2020-12-09 14:00:00,0.115997,0.000000,0.115997,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,1.0,4.0,0.0,4.0,0.0,0.0,0.0,59.266667
471,2020-12-09 15:00:00,0.070237,0.004000,0.066237,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,4.0,53.666667
472,2020-12-09 16:00:00,0.023455,0.001333,0.022121,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,52.366667


In [87]:
X,y = md.gen_feat_var(df,target = 'Downtime', features = ['Totes','Faults'])
y=1-y

In [90]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

#RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    #'criterion': 'mse', 
                                                                                    #'max_depth': None, 
                                                                                   # 'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test, fit_intercept=False)

#XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         #'max_depth': None})

#selection_options= {'threshold': -np.inf,
                    #'max_features': 180}

#X_sel_rf = md.select_features(X, y, RF_mdl, **selection_options)

#X_sel_lm = md.select_features(X, y, Linear_mdl, **selection_options)

#X_sel_XGB = md.select_features(X, y, XGB_mdl, **selection_options)

#X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,split_options = {'test_size': 0.3,
                                                                       #'random_state': None})

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X, y)

#cv_R2 = md.cross_validate_r2(RF_mdl,X, y)

#cv_R2 = md.cross_validate_r2(XGB_mdl, X_sel_XGB, y)

TypeError: run_LR_model() got an unexpected keyword argument 'fit_intercept'

### Add LM - metrics

In [91]:
X_train_rd = X_train
X = 10

def find_features(X_train, y_train, X):
    max_p = 1
    while max_p > 0.1:
        model = sm.OLS(y_train, X_train)
        results = model.fit()
        top_X = results.pvalues.sort_values(ascending=False).head(X)
        max_p = top_X.tail(1).values[0]
        rm_col = list(results.pvalues.sort_values(ascending=False).head(X).index)
        X_train = X_train.drop(rm_col, axis=1)
    return X_train.columns

In [96]:
keep_features = find_features(X_train = X_train_rd , y_train=y_train, X=X)

model = sm.OLS(y_train, X_train[keep_features])
results = model.fit()

print(len(keep_features))

cv_R2 = md.cross_validate_r2(Linear_mdl, X[keep_features], y)

print(results.summary())

#y_prediction =  results.predict(X_test)

#plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

339


TypeError: 'int' object is not subscriptable

In [18]:
fa_Both = fa[fa['Tote Colour']=="Both"]
fa_Both = fa_Both[fa_Both['Pick Station']==False]
fa_Both[fa_Both['Desk']!="Z"]['Asset Code'].value_counts()

C0801STA002    335
C0803STA112    289
C0703STA102     92
C1401ACH001     90
C0503STA251     87
C0501STA012     76
C0503STA252     73
C0502STA056     62
C0801ACH001     61
C0704STA201     47
C0903ACH113     33
C0903STA112     29
C0901STA012     28
C1003STA251     27
C1101ACH001     24
C0503ACH234     24
C1303STA102     17
C1403ACH113     13
C1103STA102     13
C1101STA012     12
C1403STA112     10
C1003STA252      9
C1401ACH011      8
C1301STA002      8
C0603STA102      7
C0904STA206      7
C0901ACH011      7
C0701STA002      6
C0503RDC101      6
C1203ACH106      6
C0703ACH101      5
C0501STA002      5
C1201STA012      5
C0503ACH233      4
C1001STA002      4
C1201ACH001      4
C0704STA206      4
C0801ACH011      4
C1403STA102      3
C0803STA102      3
C1101STA002      3
C0901RDC017      2
C0701ACH011      2
C1203ACH113      2
C1401STA012      2
C0901STA015      2
C0501ACH011      1
C0803ACH106      1
C1201ACH011      1
C1003ACH232      1
C1001STA012      1
C1001ACH011      1
C0503RDC254 

In [19]:
fa[fa['Fault ID']==7647]

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
165704,441848.0,SCS M04: single module 2 access door open fron...,2020-11-25 05:28:29,SCSM04,SCSM04,17.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165737,441848.0,SCS M04: single module 2 access door open fron...,2020-12-04 21:08:53,SCSM04,SCSM04,18.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165719,441848.0,SCS M04: single module 2 access door open fron...,2020-12-02 14:26:23,SCSM04,SCSM04,20.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165720,441848.0,SCS M04: single module 2 access door open fron...,2020-12-02 13:03:40,SCSM04,SCSM04,20.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165739,441848.0,SCS M04: single module 2 access door open fron...,2020-12-04 17:49:05,SCSM04,SCSM04,21.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165721,441848.0,SCS M04: single module 2 access door open fron...,2020-12-02 12:36:34,SCSM04,SCSM04,22.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165714,441848.0,SCS M04: single module 2 access door open fron...,2020-11-29 22:39:41,SCSM04,SCSM04,22.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165729,441848.0,SCS M04: single module 2 access door open fron...,2020-12-06 21:49:15,SCSM04,SCSM04,23.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165717,441848.0,SCS M04: single module 2 access door open fron...,2020-12-03 08:02:11,SCSM04,SCSM04,24.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165736,441848.0,SCS M04: single module 2 access door open fron...,2020-12-05 04:45:00,SCSM04,SCSM04,26.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault


In [20]:
fa[fa['Pick Station']!=False]['Asset Code'].value_counts()

C08PTT082    2458
C12PTT151    1610
C14PTT202    1568
C14PTT201    1420
C09PTT092    1359
C09PTT102    1305
C06PTT031    1110
C12PTT141     989
C09PTT091     962
C08PTT072     913
C09PTT101     895
C08PTT081     884
C13PTT181     864
C13PTT172     863
C05PTT012     840
C12PTT142     817
C14PTT192     814
C13PTT182     793
C14PTT191     776
C11PTT131     759
C13PTT171     749
C11PTT122     726
C06PTT032     718
C11PTT132     698
C07PTT041     672
C08PTT071     669
C07PTT051     661
C06PTT022     579
C12PTT152     565
C07PTT042     562
C05PTT011     528
C06PTT021     497
C10PTT112     457
C10PTT111     430
C07PTT052     410
C11PTT121     390
Name: Asset Code, dtype: int64

In [21]:
pick_stations = ['PTT011','PTT012','PTT021','PTT022','PTT031','PTT032','PTT041','PTT042','PTT051','PTT052','PTT071','PTT072','PTT081','PTT082','PTT091','PTT092','PTT101','PTT102','PTT111','PTT112','PTT121','PTT122','PTT131','PTT132','PTT141','PTT142','PTT151','PTT152','PTT171','PTT172','PTT181','PTT182','PTT191','PTT192','PTT201','PTT202']

In [22]:
fa_floor[fa_floor['Duration']==0]

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
